import libraries

Load Data

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
from sklearn import svm
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import graphviz
from sklearn import tree

In [ ]:
# players dataset 2004 - 2005 season
df_p = pd.read_csv('/content/drive/MyDrive/NBA_dataset (1)/player_regular_season.csv') 
df_players = df_p[(df_p.year == 2004)]

# teams dataset 2004 - 2005 season
df_t = pd.read_csv('/content/drive/MyDrive/NBA_dataset (1)/team_season.csv')
df_teams = df_t[((df_t.year > 1989) & (df_t.year < 2004))]
df_season2004 = df_t[((df_t.year > 2003))]
teams = df_season2004['team']

Players in the season 2004 - 2005 season

In [ ]:
# show the head of this dataset
df_players.head()

Teams in the 2004-2005 season

In [ ]:
df_teams.head()

In [ ]:
df_teams.tail()

,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_3pa,o_pts,d_fgm,d_fga,d_ftm,d_fta,d_oreb,d_dreb,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
1152,SAS,2003,N,2842,6434,1409,2069,1028,2669,3697,1676,1667,661,1155,537,408,1140,7501,2613,6388,1372,1844,909,2459,3368,1419,1837,635,1194,381,311,951,6909,90.400002,57,25
1153,SEA,2003,N,2939,6590,1363,1782,907,2318,3225,1782,1783,661,1128,387,723,1936,7964,2980,6622,1570,2028,1068,2430,3498,1728,1658,629,1158,386,486,1394,8016,91.800003,37,45
1154,TOR,2003,N,2654,6348,1237,1650,830,2419,3249,1574,1748,604,1090,401,461,1294,7006,2722,6365,1536,2055,1034,2662,3696,1518,1619,590,1137,333,273,889,7253,88.800003,33,49
1155,UTA,2003,N,2690,6171,1639,2197,1103,2272,3375,1671,2095,583,1255,494,252,786,7271,2541,5879,1874,2431,868,2131,2999,1528,1999,666,1216,530,415,1209,7371,88.099998,42,40
1156,WAS,2003,N,2758,6556,1579,2211,1115,2392,3507,1537,1712,733,1371,406,433,1269,7528,3061,6741,1444,1874,1048,2520,3568,1966,1783,787,1269,446,424,1238,7990,94.099998,25,57


Compare won vs lost

In [ ]:
# Create a column from comparing the wins vs losses
# 0 - if wins = loss
# 1 - if wins < loss
# 2 - if wins > loss
val = -1
def compare(row):
    if row['won'] < row['lost']:
        val = 1
    
    elif row['won'] > row['lost']:
        val =  2

    else:
        val = 0

    return val


Apply the changes to the dataset

In [ ]:
df_teams['teamRes'] = df_teams.apply(compare, axis=1)


In [ ]:
df_teams.head()

,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_3pa,o_pts,d_fgm,d_fga,d_ftm,d_fta,d_oreb,d_dreb,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost,teamRes
761,ATL,1990,N,3349,7223,2034,2544,1235,2420,3655,1864,1768,729,1231,374,271,0,9003,3568,7219,1587,2069,1080,2499,3579,2320,2034,688,1291,361,217,0,8940,100.802790,43,39,2
762,BOS,1990,N,3695,7214,1646,1997,1088,2697,3785,2160,1695,672,1320,565,109,0,9145,3419,7559,1639,2084,1192,2204,3396,2052,1660,738,1127,381,191,0,8668,101.146800,56,26,2
763,CHA,1990,N,3286,7033,1725,2214,1027,2200,3227,2019,1946,759,1290,304,131,0,8428,3408,6915,1884,2441,1151,2504,3665,2144,1841,675,1406,480,158,0,8858,99.808464,26,56,1
764,CHI,1990,N,3632,7125,1605,2111,1148,2342,3490,2212,1751,822,1184,438,155,0,9024,3267,6884,1554,2017,1062,2162,3224,2016,1826,633,1402,348,190,0,8278,97.917480,61,21,2
765,CLE,1990,N,3259,6857,1665,2176,1011,2329,3340,2240,1672,643,1281,450,160,0,8343,3459,7117,1464,1916,1097,2398,3495,2150,1895,710,1226,394,163,0,8545,97.749222,33,49,1


prepare x and y

In [ ]:
# o_fgm	o_fga	o_ftm	o_fta	
#o_oreb	o_dreb	o_reb	o_asts	
#o_pf	o_stl	o_to	o_blk	
#o_3pm	o_3pa	o_pts	d_fgm	
#d_fga	d_ftm	d_fta	d_oreb	
#d_dreb	d_reb	d_asts	d_pf	
#d_stl	d_to	d_blk	d_3pm	d_3pa	d_pts	pace	won	lost
feature_cols = ['o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_oreb', 'o_dreb', 
                'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_3pm',
                'o_3pa', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_oreb',
                'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk',
                'd_3pm', 'd_3pa', 'd_pts']

x = df_teams[feature_cols]
y = df_teams['teamRes']
            

In [ ]:
x.head()


,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_3pa,o_pts,d_fgm,d_fga,d_ftm,d_fta,d_oreb,d_dreb,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts
761,3349,7223,2034,2544,1235,2420,3655,1864,1768,729,1231,374,271,0,9003,3568,7219,1587,2069,1080,2499,3579,2320,2034,688,1291,361,217,0,8940
762,3695,7214,1646,1997,1088,2697,3785,2160,1695,672,1320,565,109,0,9145,3419,7559,1639,2084,1192,2204,3396,2052,1660,738,1127,381,191,0,8668
763,3286,7033,1725,2214,1027,2200,3227,2019,1946,759,1290,304,131,0,8428,3408,6915,1884,2441,1151,2504,3665,2144,1841,675,1406,480,158,0,8858
764,3632,7125,1605,2111,1148,2342,3490,2212,1751,822,1184,438,155,0,9024,3267,6884,1554,2017,1062,2162,3224,2016,1826,633,1402,348,190,0,8278
765,3259,6857,1665,2176,1011,2329,3340,2240,1672,643,1281,450,160,0,8343,3459,7117,1464,1916,1097,2398,3495,2150,1895,710,1226,394,163,0,8545


In [ ]:
y.head()

761    2
762    2
763    1
764    2
765    1
Name: teamRes, dtype: int64

split data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.20)

Prediction models

In [ ]:
# knn

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
pred = knn.predict(x_test)
print(metrics.accuracy_score(y_test, pred))
#print(knn.predict_proba(x_test))

0.7875


In [ ]:
# linear svm

clf = LinearSVC(random_state=2)
clf.fit(x_train, y_train)
pred = (clf.predict(x_test))
print(metrics.accuracy_score(y_test, pred))

0.85


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# random forest classifier

cls_forest = RandomForestClassifier()
cls_forest.fit(x_train, y_train)
pred = cls_forest.predict(x_test)
print(metrics.accuracy_score(y_test, pred))

0.825


In [ ]:
# Gradient Tree Boosting

clfgtb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clfgtb.fit(x_train, y_train)
clfgtb.score(x_test, y_test)

0.8

Apply models to the 2004 - 2005 season

In [ ]:
# prepare the data before feeding it to the models

df_currSeason = df_t[(df_t.year > 2003)]
df_currSeason['teamRes'] = df_currSeason.apply(compare, axis=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
x_new = df_currSeason[feature_cols]

y_new = df_currSeason['teamRes']

In [ ]:
# Apply knn in the 2004 - 2005 dataset
new_pred = knn.predict(x_new)
print(metrics.accuracy_score(y_new, new_pred))

0.7333333333333333


In [ ]:
# apply linear svm in the 2004 - 2005

new_pred = (clf.predict(x_new))
print(metrics.accuracy_score(y_new, new_pred))

0.8666666666666667


In [ ]:
# apply random forest classifier in the 2004 - 2005

new_pred = cls_forest.predict(x_new)
print(metrics.accuracy_score(y_new, new_pred))

0.7666666666666667


In [ ]:
# apply Gradient Tree Boosting
clfgtb.score(x_new, y_new)

0.8

In [ ]:
df_season2004.head()

,team,year,leag,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_3pa,o_pts,d_fgm,d_fga,d_ftm,d_fta,d_oreb,d_dreb,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts,pace,won,lost
1157,ATL,2004,N,2942,6672,1417,1994,1100,2335,3435,1614,2009,629,1319,344,304,973,7605,3044,6389,1846,2396,902,2483,3385,1803,1777,716,1235,484,467,1232,8401,93.948730,13,69
1158,BOS,2004,N,3046,6511,1775,2323,909,2438,3347,1810,2000,667,1297,423,437,1252,8304,2960,6664,1825,2423,1039,2427,3466,1859,1937,714,1248,402,488,1370,8233,95.735855,45,37
1159,CHI,2004,N,2849,6592,1536,2048,1001,2591,3592,1743,2028,576,1371,369,511,1433,7745,2763,6548,1716,2305,979,2524,3503,1715,1914,685,1272,434,416,1245,7658,96.283737,47,35
1160,CHR,2004,N,2961,6859,1487,2096,1083,2335,3418,1794,1893,695,1192,440,320,881,7729,3015,6507,1748,2279,959,2586,3545,1768,1857,624,1304,446,442,1224,8220,96.201630,18,64
1161,CLE,2004,N,2990,6687,1634,2174,1117,2352,3469,1851,1850,654,1141,461,300,904,7914,2913,6444,1601,2135,938,2324,3262,1737,1770,549,1226,419,422,1122,7849,93.840187,42,40


In [ ]:
x_new.head()

,o_fgm,o_fga,o_ftm,o_fta,o_oreb,o_dreb,o_dreb,o_reb,o_asts,o_pf,o_stl,o_to,o_blk,o_3pm,o_3pa,o_pts,d_fgm,d_fga,d_ftm,d_fta,d_oreb,d_dreb,d_reb,d_asts,d_pf,d_stl,d_to,d_blk,d_3pm,d_3pa,d_pts
1157,2942,6672,1417,1994,1100,2335,2335,3435,1614,2009,629,1319,344,304,973,7605,3044,6389,1846,2396,902,2483,3385,1803,1777,716,1235,484,467,1232,8401
1158,3046,6511,1775,2323,909,2438,2438,3347,1810,2000,667,1297,423,437,1252,8304,2960,6664,1825,2423,1039,2427,3466,1859,1937,714,1248,402,488,1370,8233
1159,2849,6592,1536,2048,1001,2591,2591,3592,1743,2028,576,1371,369,511,1433,7745,2763,6548,1716,2305,979,2524,3503,1715,1914,685,1272,434,416,1245,7658
1160,2961,6859,1487,2096,1083,2335,2335,3418,1794,1893,695,1192,440,320,881,7729,3015,6507,1748,2279,959,2586,3545,1768,1857,624,1304,446,442,1224,8220
1161,2990,6687,1634,2174,1117,2352,2352,3469,1851,1850,654,1141,461,300,904,7914,2913,6444,1601,2135,938,2324,3262,1737,1770,549,1226,419,422,1122,7849


Predict games 

In [ ]:
#print(teams)
# Teams in the 2004 - 2005 season
# ATL  BOS CHI CHR CLE DAL DEN DET GSW NOH
# HOU IND LAC LAL MEM MIA MIL MIN NJN NYK ORL
# PHI PHO POR SAC SAS SEA TOR UTA WAS

allTeams = ['ATL', 'BOS', 'CHI', 'CHR', 'CLE', 'DA', 'DEN', 'DET', 'GSW',
            'NOH', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN',
            'NJN', 'NYK', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'SEA', 
            'TOR', 'UTA', 'WAS']
            
# get the data for each team
teamsList = []
fromDF = []


# we want the data for the opposition team first
for index, row in x_new.iterrows():
    fromDF.append(row)

for i in range(len(fromDF)):
    team = []
    for j in range(15, 30):
        team.append(fromDF[i][j])

    teamsList.append(team)



In [ ]:
print(teamsList[28])

[2731, 5965, 2027, 2653, 859, 2230, 3089, 1592, 2057, 646, 1213, 474, 486, 1297, 7975]


In [ ]:
# predict the games between two teams

#games
games = ['PHX1 vs SAS2',
         'DET1 vs PHI2', 
         'MIN1 vs BOS2',
         'NY1 vs MIA2', 
         'TOR1 vs MIL2', 
         'CHI1 vs DAL2', 
         'UTA1 vs DEN2', 
         'WSH1 vs MEM2', 
         'ATL1 vs POR2', 
         'CHA1 vs LAL2']

game1 = []   # contains the home team vs away team

In [ ]:
# I will demonstrate fpr one game, the logic for all other games is the same

# PHI vs SAS
i = allTeams.index('PHI')
j = allTeams.index('SAS')

team1 = teamsList[i]
team2 = teamsList[j]
teamsCombined = team1 + team2
game1.append(teamsCombined)

In [ ]:
len(x_new.columns)

30

In [ ]:
len(game1[0])

30

In [ ]:
# game PHI vs SAS prediction using knn
# 0 - draw
# 1 - home team losses
# 2 - home team wins
pred1 = knn.predict(game1)
prob1 = knn.predict_proba(game1)
print(pred1)
print(prob1)

[2]
[[0. 0. 1.]]


In [ ]:
#  game PHI vs SAS prediction using linear svm

pred2 = clf.predict(game1)
print(pred2)

[2]


In [ ]:
#  game PHI vs SAS prediction using forest classifier 

pred3 = cls_forest.predict(game1)
prob3 = cls_forest.predict_proba(game1)
print(pred3)
print(prob3)

In [ ]:
# game PHI vs SAS prediction using Gradient Tree Boosting

pred4 = clfgtb.predict(game1)
prob4 = clfgtb.predict_proba(game1)
print(pred4)
print(prob4)

[2]
[[6.90224237e-07 4.49668214e-09 9.99999305e-01]]
